## Tools & Agents

In [2]:
## Arxiv - Research paper 
# Tool creation 
from langchain_community.tools import ArxivQueryRun , WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper , ArxivAPIWrapper

In [3]:
# Used the inbuilt tool of wikipedia
api_wraper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wraper_wiki)
wiki.name

'wikipedia'

In [5]:
api_wraper_arxiv = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper = api_wraper_arxiv)
arxiv.name

'arxiv'

In [6]:
tools = [wiki,arxiv]

In [8]:
## Custom tools [RAG tools]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os

load_dotenv()

os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [9]:
loader = WebBaseLoader('https://docs.smith.langchain.com/')
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents , embedding)
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x781dd66f55d0>, search_kwargs={})

In [10]:
## Convert retrival into tools 
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith-search","search any infromation about langsmith")
retriever_tool.name

'langsmith-search'

In [11]:
tools  = [wiki , arxiv , retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/omkar/anaconda3/envs/mlenv/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='search any infromation about langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x781dd42d6170>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x781dd66f55d0>, s

In [12]:
# Run all the tool with agents and llm 

## Tools , LLM => AgentExecutor 

from langchain_groq import ChatGroq
os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")
groq_api_key=os.getenv("GROQ_API_KEY")

llm = ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

In [13]:
## Prompt template 

from langchain import hub 
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [14]:
## Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [15]:
## Agent Executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent = agent, tools = tools, verbose = True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [16]:
agent_executor.invoke({"input": "tell me about langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
ObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based on these.EvalsEvaluate your application over production traffic — score application performance and get human feedback on your data.Prompt EngineeringIterate on prompts, with automatic version control and collaboration features.
LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with Lan

{'input': 'tell me about langsmith',
 'output': 'It appears that LangSmith is a platform for building production-grade Large Language Model (LLM) applications. It allows developers to closely monitor and evaluate their applications, enabling them to ship quickly and with confidence. LangSmith provides features such as observability, evaluation, and prompt engineering, which can be useful for debugging and improving LLM applications.'}

In [17]:
agent_executor.invoke({"input": "What is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witBased on the result, it seems that machine learning is a subfield of artificial intelligence that deals with developing algorithms that can learn from data and make predictions or decisions without being explicitly programmed. This is in contrast to traditional rule-based systems, which rely on hand-crafted rules to make decisions.

Machine learning involves training algorithms on large datasets, which enables them to learn patterns and relationships within the data. The algorithm can then use this knowledge to make predictions or decisions on new, unseen data.

Some common applications of machine learning include image and speech recogniti

{'input': 'What is machine learning',
 'output': 'Based on the result, it seems that machine learning is a subfield of artificial intelligence that deals with developing algorithms that can learn from data and make predictions or decisions without being explicitly programmed. This is in contrast to traditional rule-based systems, which rely on hand-crafted rules to make decisions.\n\nMachine learning involves training algorithms on large datasets, which enables them to learn patterns and relationships within the data. The algorithm can then use this knowledge to make predictions or decisions on new, unseen data.\n\nSome common applications of machine learning include image and speech recognition, natural language processing, recommender systems, and predictive analytics. Machine learning has many potential benefits, such as improving accuracy, reducing costs, and increasing efficiency.\n\nWould you like to know more about a specific aspect of machine learning, or would you like to ask 

In [18]:
agent_executor.invoke({"input": "What is the paper 1605.08386 about ?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
allowed moves of arbitrary length. We show that the diThe paper 1605.08386 is titled "Heat-bath random walks with Markov bases" and was published in 2016 by Caprice Stanley and Tobias Windisch. The summary of the paper is about studying graphs on lattice points with edges coming from a finite set of allowed moves of arbitrary length, and showing that the dynamics of these graphs can be described by a Markov process.

> Finished chain.


{'input': 'What is the paper 1605.08386 about ?',
 'output': 'The paper 1605.08386 is titled "Heat-bath random walks with Markov bases" and was published in 2016 by Caprice Stanley and Tobias Windisch. The summary of the paper is about studying graphs on lattice points with edges coming from a finite set of allowed moves of arbitrary length, and showing that the dynamics of these graphs can be described by a Markov process.'}